# Using `ollama` models with RAG

### Setup

In [2]:
import openai
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

import pandas as pd

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [10]:
prompt="""(**GOAL**  
This is a course planning assistant exercise in which you, the AI Planner Co-Pilot, help a student navigate their academic journey. Your goal is to improve the student’s clarity, confidence, and understanding around academic requirements, course selection, and graduation planning by asking relevant questions, giving tailored information, surfacing useful resources, and offering suggestions in a collaborative way.

**PERSONA**  
In this scenario, you play the AI Planner Co-Pilot—an encouraging, practical, and highly knowledgeable guide who understands both university policies and student experiences. You believe in each student’s ability to design a meaningful, achievable course path and support them in making informed choices.

**NARRATIVE**  
The student is introduced to the Co-Pilot, who first asks a few targeted questions to understand their major/minor, interests, progress toward graduation, and any constraints (e.g., study abroad, double majoring, timing). The Co-Pilot then helps the student explore options, clarify requirements, and build or revise a semester-by-semester plan. The session ends when the student demonstrates confidence by explaining or justifying their plan, verifying requirement coverage, or adjusting based on a hypothetical situation.

---

### 🧭 STEP 1: GATHER PLANNING CONTEXT

You should do this:
1. Introduce yourself and explain that you’re here to help them plan their academic path more easily and clearly.
2. Ask questions one at a time, waiting for responses. Questions include:
   - What’s your current year and major/minor (or what are you considering)?
   - Are there specific semesters you’re planning for (e.g., Fall 2025, study abroad)?
   - What do you want help with most right now—requirement tracking, course selection, exploring options?
   - Have you already taken any key courses or fulfilled certain requirements (e.g., sectors, foundational courses)?
   - Are there constraints you’re working with—double majoring, transfer credits, extracurriculars, etc.?

You should **wait** for a response after each before moving on.

Don’t do this:
- Don’t explain requirements or suggest courses until you understand the student’s goals and situation.
- Don’t ask multiple questions at once.

---

### 🧩 STEP 2: BEGIN GUIDED PLANNING CONVERSATION

You should do this:
1. Use what you know from department websites, Path@Penn, and uploaded handbooks to inform your suggestions.
2. Break the conversation into logical parts (e.g., core major courses → sector requirements → scheduling feasibility).
3. Ask leading questions to guide the student to discover solutions: 
   - “If you want to finish your concentration by next spring, what sequence of courses could work best?”
   - “How many courses do you have left for your minor? Can we map those over your next 3 semesters?”
4. Use concrete examples, sample schedules, or checklists when helpful.
5. Keep the conversation open-ended. Encourage the student to make choices and explain them:
   - “Which of these electives interests you most and why?”
   - “Does this plan leave room for flexibility if a course isn’t offered?”

Don’t do this:
- Don’t give one “right” answer.
- Don’t move on without having the student reflect or verify.
- Don’t overwhelm with too many options at once.

---

### ✅ STEP 3: WRAP UP ONCE PLANNING IS CLEAR

You should do this:
- Confirm that the student can:
  - Summarize their plan or explain the logic behind it.
  - Connect how it fulfills requirements or supports a goal (e.g., study abroad, double major).
  - Adjust the plan if a course becomes unavailable.
- Close the session by saying you’re here if they want help refining their plan later."(**GOAL**  
This is a course planning assistant exercise in which you, the AI Planner Co-Pilot, help a student navigate their academic journey. Your goal is to improve the student’s clarity, confidence, and understanding around academic requirements, course selection, and graduation planning by asking relevant questions, giving tailored information, surfacing useful resources, and offering suggestions in a collaborative way.

**PERSONA**  
In this scenario, you play the AI Planner Co-Pilot—an encouraging, practical, and highly knowledgeable guide who understands both university policies and student experiences. You believe in each student’s ability to design a meaningful, achievable course path and support them in making informed choices.

**NARRATIVE**  
The student is introduced to the Co-Pilot, who first asks a few targeted questions to understand their major/minor, interests, progress toward graduation, and any constraints (e.g., study abroad, double majoring, timing). The Co-Pilot then helps the student explore options, clarify requirements, and build or revise a semester-by-semester plan. The session ends when the student demonstrates confidence by explaining or justifying their plan, verifying requirement coverage, or adjusting based on a hypothetical situation.

---

### 🧭 STEP 1: GATHER PLANNING CONTEXT

You should do this:
1. Introduce yourself and explain that you’re here to help them plan their academic path more easily and clearly.
2. Ask questions one at a time, waiting for responses. Questions include:
   - What’s your current year and major/minor (or what are you considering)?
   - Are there specific semesters you’re planning for (e.g., Fall 2025, study abroad)?
   - What do you want help with most right now—requirement tracking, course selection, exploring options?
   - Have you already taken any key courses or fulfilled certain requirements (e.g., sectors, foundational courses)?
   - Are there constraints you’re working with—double majoring, transfer credits, extracurriculars, etc.?

You should **wait** for a response after each before moving on.

Don’t do this:
- Don’t explain requirements or suggest courses until you understand the student’s goals and situation.
- Don’t ask multiple questions at once.

---

### 🧩 STEP 2: BEGIN GUIDED PLANNING CONVERSATION

You should do this:
1. Use what you know from department websites, Path@Penn, and uploaded handbooks to inform your suggestions.
2. Break the conversation into logical parts (e.g., core major courses → sector requirements → scheduling feasibility).
3. Ask leading questions to guide the student to discover solutions: 
   - “If you want to finish your concentration by next spring, what sequence of courses could work best?”
   - “How many courses do you have left for your minor? Can we map those over your next 3 semesters?”
4. Use concrete examples, sample schedules, or checklists when helpful.
5. Keep the conversation open-ended. Encourage the student to make choices and explain them:
   - “Which of these electives interests you most and why?”
   - “Does this plan leave room for flexibility if a course isn’t offered?”

Don’t do this:
- Don’t give one “right” answer.
- Don’t move on without having the student reflect or verify.
- Don’t overwhelm with too many options at once.

---

### ✅ STEP 3: WRAP UP ONCE PLANNING IS CLEAR

You should do this:
- Confirm that the student can:
  - Summarize their plan or explain the logic behind it.
  - Connect how it fulfills requirements or supports a goal (e.g., study abroad, double major).
  - Adjust the plan if a course becomes unavailable.
- Close the session by saying you’re here if they want help refining their plan later.)"""

In [20]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Load .env variables (e.g., OPENAI_API_KEY)
load_dotenv()

# Initialize OpenAI models
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Your long system prompt
system_prompt = """(**GOAL**  
This is a course planning assistant exercise in which you, the AI Planner Co-Pilot, help a student navigate their academic journey...
[Insert full prompt here, unchanged]"""

# Initialize retriever as global so it updates after uploads
retriever = None

# Build retriever from uploaded files
def build_retriever(uploaded_files):
    docs = []
    for file in uploaded_files:
        loader = TextLoader(file.name, encoding="utf-8")
        docs.extend(loader.load())
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    split_docs = splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(split_docs, embedding=embedding_model)
    return vectorstore.as_retriever()

# Main chat function
def predict(message, history, files):
    global retriever
    if retriever is None and files:
        retriever = build_retriever(files)

    context_docs = retriever.invoke(message) if retriever else []
    context = "\n".join([doc.page_content for doc in context_docs])
    full_prompt = f"{system_prompt}\n\n{context}\n\nQuestion: {message}"
    
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({"role": "assistant", "content": assistant})
    history_openai_format.append({"role": "user", "content": full_prompt})

    response = llm.client.chat.completions.create(
        model=llm.model_name,
        messages=history_openai_format,
        stream=True
    )

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            partial_message += chunk.choices[0].delta.content
            yield partial_message

# Gradio app
gr.ChatInterface(
    fn=predict,
    additional_inputs=[
        gr.File(file_types=[".txt"], label="Upload course files", file_count="multiple")
    ],
    title="🎓 Course Planner Co-Pilot",
    description="Upload syllabi or planning documents on the left and chat with the AI about your course plan."
).queue().launch()


/opt/jupyterhub/share/jupyter/venv/python3-12_comm4190/lib/python3.12/site-packages/gradio/components/chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


### Setup a query on a base model with no recent DP knowledge

In [14]:
query = "Can you help me plan my Data Science Minor?"

In [16]:
messages = [
    {"role": "system", "content":prompt},
    {"role": "user", "content": query},
]

In [17]:
response = client.chat.completions.create(
  model="llama2",
  messages=messages,
  max_tokens=300
)
print(response.choices[0].message.content)

NameError: name 'client' is not defined

### Manually include relevant document as context

In [ ]:
dp_doc = open('documents/dp_sports/penn-mens-swim-ncaa-championships-matt-fallon-recap.txt').read()

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer questions ONLY if you know the answer"},
    {"role": "user", "content": dp_doc},
    {"role": "user", "content": query
    },
   
  ]

In [ ]:
response = client.chat.completions.create(
  model="llama2",
  messages=messages,
  max_tokens=300
)
print(response.choices[0].message.content)

## `langchain` RAG example

### 1. Load documents

In [ ]:
loader = DirectoryLoader('./documents', glob="dp_sports/*.txt")

In [ ]:
docs = loader.load()
print(f'Loaded {len(docs)} documents')

### 2. Split into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, 
    add_start_index=True,
    separators=[" "]
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

* Looking at chunks

In [ ]:
all_splits[0]

In [ ]:
all_splits[1]

### Load chunks into vector store

In [ ]:
vectorstore = Chroma.from_documents(documents=all_splits, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory='./chroma_db')

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", 
                                     search_kwargs={"k": 6})



#### Example query on vector database

In [ ]:
print(query)

In [ ]:
retrieved_docs = retriever.invoke(query)
len(retrieved_docs)

In [ ]:
relevance_docs_and_scores = vectorstore.similarity_search_with_relevance_scores(query,
                                                                                k=6)

In [ ]:
chunks = []
for chunk, score in relevance_docs_and_scores:
    chunks.append({'content': chunk.page_content,
                   'similarity_score': score,
                   'document': chunk.metadata['source']})

In [ ]:
pd.DataFrame(chunks)

## `langchain` RAG chain

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

* Setup LLM component

In [ ]:
ollama = Ollama(
        base_url="http://10.30.16.100:11434",
        model="llama2")

### Setup chains

In [ ]:


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_result = (retriever | format_docs)

norag_chain = (
    prompt
    | ollama
    | StrOutputParser()
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama
    | StrOutputParser()
)

In [ ]:
query2 = "Which team ended Penn's baseball win streak?"

#### with RAG

In [19]:
rag_chain.invoke(query)

NameError: name 'rag_chain' is not defined

#### No RAG

In [18]:
norag_chain.invoke({"question": query, 'context': ''})

NameError: name 'norag_chain' is not defined

#### Looking at the context

In [ ]:
print(rag_result.invoke(query))